## 실습 및 데이터셋 소개

1. 간단한 word2vec 모델 활용 및 embedding을 활용한 계산 법 익히기
2. 다양한 embedding 모델을 실제 활용하면서 현재 데이터의 retrieval 성능 체크
3. Search 이외에도 embedding을 활용할 수 있는 다양한 방법 소개
4. Embedding을 활용한 간단한 서비스 구현 (search 최적화)
---

# 1. The Simpsons dataset

#### 데이터 소개 : 심슨 등장 인물들의 대화를 담은 데이터 셋
#### 데이터 활용 목적 : 대화 분석을 통해 단어들간의 관계를 파악

![](https://images.edrawmax.com/what-is/simpsons-family-tree/example.png) <br>
출처 : https://images.edrawmax.com/what-is/simpsons-family-tree/example.png

- 다운로드 : https://www.kaggle.com/datasets/pierremegret/dialogue-lines-of-the-simpsons?resource=download

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('simpsons_dataset.csv')
df.shape

In [ ]:
df.head()

In [ ]:
from collections import Counter

In [ ]:
counts = Counter(df['raw_character_text'])

In [ ]:
counts.most_common()

# 2. Quora dataset

<img src="https://images.unsplash.com/photo-1649180573784-bd23dbe5606a?q=80&w=2062&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D" width="500" height="300"/>
<br>
출처 : https://images.unsplash.com/photo-1649180573784-bd23dbe5606a?q=80&w=2062&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D

#### 데이터 소개 : 네이버의 지식IN과 비슷한 목적을 가진 플랫폼인 Quora에서, 유사한 질문들을 모아둔 데이터 셋.
#### 데이터 활용 목적 : Embedding을 기반으로 유사한 질문을 탐색하는 실습에 활용

- datasets 패키지 활용

In [ ]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("quora")

In [ ]:
dataset.keys()

In [ ]:
raw_df = dataset["train"].to_pandas()

In [ ]:
raw_df.head()

In [ ]:
raw_df.loc[0]['questions']

중복된 질문이라고 체크된 질문들만 선택

In [ ]:
raw_df = raw_df.loc[raw_df['is_duplicate']==True].reset_index(drop=True)

In [ ]:
raw_df.loc[0, 'questions']

In [ ]:
# 중복되는 id를 개별 컬럼으로 배치
raw_df["q1"] = raw_df["questions"].apply(lambda q: q["text"][0])
raw_df["q2"] = raw_df["questions"].apply(lambda q: q["text"][1])
raw_df["id1"] = raw_df["questions"].apply(lambda q: q["id"][0])
raw_df["id2"] = raw_df["questions"].apply(lambda q: q["id"][1])

q1_to_q2 = raw_df.copy().rename(columns={"q1": "text", "id1": "id", "id2": "dq_id"}).drop(columns=["questions", "q2"])
q2_to_q1 = raw_df.copy().rename(columns={"q2": "text", "id2": "id", "id1": "dq_id"}).drop(columns=["questions", "q1"])
flat_df = pd.concat([q1_to_q2, q2_to_q1])

flat_df = flat_df.sort_values(by=['id']).reset_index(drop=True)

In [ ]:
flat_df.head()

In [ ]:
flat_df.loc[flat_df['id']==568]

전체 데이터 중 작은 샘플만 활용

In [ ]:
flat_df = flat_df.loc[((flat_df['id'] <= 15000) & (flat_df['dq_id'] <= 15000))]

In [ ]:
flat_df.shape

In [ ]:
flat_df.head()

In [ ]:
# 각 질문 하나당 중복되는 질문 id를 list 형태로 저장
df = flat_df.drop_duplicates("id")
df.loc[:, "duplicated_questions"] = df["id"].apply(lambda qid: flat_df[flat_df["id"] == qid]["dq_id"].tolist())
df = df.drop(columns=["dq_id", "is_duplicate"])
df.loc[:, 'length'] = [len(x) for x in df['duplicated_questions']]

df.head()

In [ ]:
df.loc[[len(i)>2 for i in df.duplicated_questions]]

In [ ]:
df.to_csv("quora_dataset.csv", index=False)

In [ ]:
df.shape

# 3. ABC News dataset

<img src="https://s.abcnews.com/images/US/abc_news_default_2000x2000_update_16x9_992.jpg" width="500" height="300"/>

출처 : https://s.abcnews.com/images/US/abc_news_default_2000x2000_update_16x9_992.jpg

- 다운로드 : https://www.kaggle.com/datasets/therohk/million-headlines

#### 데이터 소개 : ABC news의 날짜와 헤드라인
#### 데이터 활용 목적 : Embedding을 다양한 machine learning 모델들과 결합하여 정보를 처리하는 방법 소개

In [ ]:
df = pd.read_csv("abcnews.csv")

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.publish_date.max(), df.publish_date.min()

In [ ]:
news_2020 = df.loc[(df['publish_date']>=20200101) & (df['publish_date']<20200201)].reset_index(drop=True)

In [ ]:
news_2020.head()

In [ ]:
news_2020.tail()

In [ ]:
news_2020.to_csv("abcnews_2020.csv", index=False)

# 4. Resume data

<img src="https://images.unsplash.com/photo-1698047681432-006d2449c631?q=80&w=2070&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D" width="500" height="300"/>

- 출처 : https://images.unsplash.com/photo-1698047681432-006d2449c631?q=80&w=2070&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D
- 다운로드 : https://www.kaggle.com/datasets/snehaanbhawal/resume-dataset/data

#### 데이터 소개 : livecareer.com 에서 발췌한 예시 resume
#### 데이터 활용 목적 : Search 최적화를 통한 간단한 서비스 구현. 우리가 요구하는 스킬과 경험을 갖고 있는 후보 search!

In [ ]:
resume = pd.read_csv("resume/Resume.csv")

In [ ]:
resume.head()

In [ ]:
resume.Category.unique()

--END--